<a href="https://colab.research.google.com/github/MocT117/Another-one-/blob/master/version1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import re
from datetime import datetime

# ----------------------------- #
# 1. Parámetros y configuración #
# ----------------------------- #
INPUT_FOLDER = 'raw_data/'
OUTPUT_FOLDER = 'output/'

# --------------------------- #
# 2. Función para extraer MAD #
# --------------------------- #
def extraer_fechas_mad(texto):
    """
        Extrae fechas tipo MAD o NEW MAD de un texto y las normaliza a formato YYYY-MM-DD.
            """
                if not isinstance(texto, str):
                        return []

                            patron = r'(?:MAD|NEW MAD)[:\s]*([0-9]{1,2}[./][0-9]{1,2}(?:[./][0-9]{2,4})?)'
                                matches = re.findall(patron, texto, flags=re.IGNORECASE)

                                    fechas_limpias = []
                                        for fecha in matches:
                                                try:
                                                            fecha = fecha.replace('/', '.')
                                                                        partes = fecha.split('.')
                                                                                    if len(partes) == 2:
                                                                                                    dia, mes = partes
                                                                                                                    año = '2025'
                                                                                                                                elif len(partes) == 3:
                                                                                                                                                dia, mes, año = partes
                                                                                                                                                                if len(año) == 2:
                                                                                                                                                                                    año = '20' + año
                                                                                                                                                                                                else:
                                                                                                                                                                                                                continue
                                                                                                                                                                                                                            fecha_formateada = datetime(int(año), int(mes), int(dia)).strftime('%Y-%m-%d')
                                                                                                                                                                                                                                        fechas_limpias.append(fecha_formateada)
                                                                                                                                                                                                                                                except Exception as e:
                                                                                                                                                                                                                                                            print(f"⚠️ Error al convertir '{fecha}' → {e}")
                                                                                                                                                                                                                                                                        continue

                                                                                                                                                                                                                                                                            return fechas_limpias

                                                                                                                                                                                                                                                                            # -------------------------------------------- #
                                                                                                                                                                                                                                                                            # 3. Función para obtener última MAD y cambios #
                                                                                                                                                                                                                                                                            # -------------------------------------------- #
                                                                                                                                                                                                                                                                            def obtener_mad_y_cambios(row):
                                                                                                                                                                                                                                                                                """
                                                                                                                                                                                                                                                                                    Devuelve la fecha MAD más reciente y el número de cambios detectados por fila.
                                                                                                                                                                                                                                                                                        Si no hay MAD en el texto, toma la de 'Material staging'.
                                                                                                                                                                                                                                                                                            """
                                                                                                                                                                                                                                                                                                fechas = extraer_fechas_mad(row['Text sales order'])
                                                                                                                                                                                                                                                                                                    # Si no hay fechas MAD en el texto, usar Material staging (y normalizarla)
                                                                                                                                                                                                                                                                                                        if not fechas or all([not f for f in fechas]):
                                                                                                                                                                                                                                                                                                                fecha_staging = row.get('Material staging')
                                                                                                                                                                                                                                                                                                                        try:
                                                                                                                                                                                                                                                                                                                                    # Normalizar fecha staging si es datetime o string
                                                                                                                                                                                                                                                                                                                                                if isinstance(fecha_staging, pd.Timestamp):
                                                                                                                                                                                                                                                                                                                                                                return fecha_staging.strftime('%Y-%m-%d'), 0
                                                                                                                                                                                                                                                                                                                                                                            elif isinstance(fecha_staging, str) and fecha_staging.strip() != '':
                                                                                                                                                                                                                                                                                                                                                                                            # Permitir fechas tipo '10/28/2025', '2025-10-28', etc.
                                                                                                                                                                                                                                                                                                                                                                                                            fecha = pd.to_datetime(fecha_staging, errors='coerce')
                                                                                                                                                                                                                                                                                                                                                                                                                            if pd.notnull(fecha):
                                                                                                                                                                                                                                                                                                                                                                                                                                                return fecha.strftime('%Y-%m-%d'), 0
                                                                                                                                                                                                                                                                                                                                                                                                                                                        except Exception as e:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                    print(f"Error con fecha staging: {e}")
                                                                                                                                                                                                                                                                                                                                                                                                                                                                            return None, 0

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                # Si hay varias fechas, usamos la más reciente y contamos cambios
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    fechas = sorted(set(fechas))
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        ultima_fecha = fechas[-1]
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            cambios = len(fechas) - 1
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                return ultima_fecha, cambios

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                # ---------------------------------------------------- #
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                # 4. Función para crear la Key primaria compuesta      #
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                # ---------------------------------------------------- #
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                def crear_columna_key(df):
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    df = df.copy()
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        df['SalesOrder_clean'] = df['Sales order no.'].astype(str).str.lstrip('0')
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            df['MaterialNumber_clean'] = df['Material number'].astype(str).str.lstrip('0')
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                df['Key'] = df['SalesOrder_clean'] + '_' + df['MaterialNumber_clean']
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    return df

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    # ----------------------------------------------- #
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    # 5. Función principal de procesamiento del flujo #
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    # ----------------------------------------------- #
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    def main():
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        os.makedirs(OUTPUT_FOLDER, exist_ok=True)

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            # Leer todos los archivos .xlsx de raw_data/
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                archivos = [f for f in os.listdir(INPUT_FOLDER) if f.endswith('.xlsx')]
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    df_total = pd.DataFrame()
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        for archivo in archivos:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                ruta = os.path.join(INPUT_FOLDER, archivo)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        try:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    df = pd.read_excel(ruta, engine='openpyxl')
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                df_total = pd.concat([df_total, df], ignore_index=True)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        except Exception as e:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    print(f"Error al leer {archivo}: {e}")

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        # Validar columnas requeridas
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            columnas_esperadas = ['Sales order no.', 'Material number', 'Text sales order', 'Material staging']
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                for col in columnas_esperadas:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        if col not in df_total.columns:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    raise ValueError(f"Falta la columna requerida: {col}")

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        # Crear columna Key
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            df_total = crear_columna_key(df_total)

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                # Aplicar función de MAD y cambios
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    df_total[['Ultima_Fecha_MAD', 'Cambios_MAD']] = df_total.apply(
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            lambda row: pd.Series(obtener_mad_y_cambios(row)),
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    axis=1
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        )

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            # Guardar resultado
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                nombre_archivo = f'consolidado_{datetime.today().strftime("%Y%m%d")}.xlsx'
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    ruta_salida = os.path.join(OUTPUT_FOLDER, nombre_archivo)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        df_total.to_excel(ruta_salida, index=False)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            print(f"✅ Archivo exportado: {ruta_salida}")

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            if __name__ == '__main__':
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                main()
